In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
loader = DirectoryLoader('opinion_text', glob="*.txt",loader_cls=TextLoader)

docs = loader.load()


In [2]:
doc_sources = [doc.metadata['source']  for doc in docs]
doc_sources

['opinion_text/9481022_In the Matter of Shaquandra A. Woods.txt',
 'opinion_text/9505041_JANE DOE v. STABLEGOLD HOSPITALITY, LLC.txt',
 'opinion_text/9506609_WILLIAM COLE REDFEARN v. JAADE MOORE.txt',
 'opinion_text/9481023_In the Matter of Andrea Jo Anne David-Vega.txt',
 'opinion_text/9481025_CRARY v. CLAUTICE.txt',
 'opinion_text/9506056_In the Interest of I.H., a Child (Mother).txt',
 'opinion_text/9501548_Ty Rutledge v. State.txt',
 'opinion_text/1327652_Turner v. Turner.txt',
 'opinion_text/9506575_Bibb County School District v. Ivori Bennett.txt',
 'opinion_text/9501681_Feder v. State.txt',
 'opinion_text/9503118_RAFAEL E. AGUIRRE v. YESLY TAVERA.txt',
 'opinion_text/9502281_Vincent Ivory Hall v. State.txt',
 'opinion_text/9507128_Ptav, Inc. v. Ralph + Rita Venture, LLC.txt',
 'opinion_text/9505463_Mayor and Aldermen of the City of Savannah v. Gladys Charlene D. Altman.txt',
 'opinion_text/9505827_Greene County Health Care, LLC v. Janice Heath.txt',
 'opinion_text/9502286_Andrew

In [3]:
import os


os.environ['OPENAI_API_KEY'] = 'sk-proj-aEREtcEjUj6xS7dx6qWlT3BlbkFJGJSTclsKd1CBzSPc9FOq'
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
documents = text_splitter.split_documents(docs)
db = Chroma.from_documents(persist_directory='./chromadb',documents=documents,embedding=OpenAIEmbeddings(model="text-embedding-3-large"))


In [11]:
from langchain.chains.question_answering import load_qa_chain
import json
from langchain.chains.conversational_retrieval.prompts import (
    CONDENSE_QUESTION_PROMPT,

)
from langchain.chains import (
    LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
def chat_scale_ai(query):
    prompt_template = """
                Act as an expert lawyer and use the following pieces of context of cases to formulate an answer based on them. If you don't know the answer, just say that you don't know, don't try to make up an answer.
                {context}
                Question: {question}
                Helpful Answer:"""
    try:
        with open('chat_history.json', 'r') as chat_hist_load:
                chat_hist_dict = json.load(chat_hist_load)
                chat_hist_dict_for_llm = [(x,y) for x,y in chat_hist_dict['Question']] 
        if len(chat_hist_dict) > 2:
            chat_hist_dict = {}
            with open('chat_history.json', 'w') as chat_hist_load:
                chat_hist_dict = json.dump({})      
    except:
        chat_hist_dict = {}
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    loaded_db = Chroma(persist_directory='./chromadb', embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))
    llm = ChatOpenAI(model_name='gpt-4o',temperature=0.1)
    question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
    doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)
    test_retriever = loaded_db.as_retriever()
    qa = ConversationalRetrievalChain(
        combine_docs_chain=doc_chain,
        retriever=test_retriever,
        question_generator=question_generator,
        
    )

    if len(chat_hist_dict) > 0:
        payload = qa({"question":str(query), "chat_history":chat_hist_dict_for_llm})
    else:
        payload = qa({"question":str(query), "chat_history":[]})
        
        
    if len(chat_hist_dict) == 0:
        chat_hist_dict["Question"] = []

    chat_hist_dict['Question'].append((query, payload['answer']))

    with open('chat_history.json', 'w') as chat_hist:
        json.dump(chat_hist_dict,chat_hist)
    # print(payload['answer'])
    return payload['answer']

In [12]:
chat_scale_ai("What is the standard for determining child custody in Georgia?")

'In Georgia, the standard for determining child custody is the "best interests of the child." This standard is codified in OCGA § 19-9-3(a)(2), which requires the court to consider the best interests of the child in making custody decisions. The court evaluates various factors to determine what arrangement will best serve the child\'s physical, mental, and emotional needs. These factors can include, but are not limited to:\n\n1. The love, affection, bonding, and emotional ties existing between each parent and the child.\n2. The capacity and disposition of each parent to give the child love, affection, and guidance and to continue the education and rearing of the child.\n3. Each parent\'s knowledge and familiarity of the child and the child\'s needs.\n4. The capacity and disposition of each parent to provide the child with food, clothing, medical care, day-to-day needs, and other necessary basic care.\n5. The home environment of each parent considering the promotion of nurturance and sa